Import UK data

Healed data are missing - Actual cases can't be calculated

In [11]:
import pandas as pd
import glob
import os

#get path for os 
path = os.getcwd() + "/data_source/UK/covid-19-uk-data-master/data/covid-19-indicators-uk.csv"
path = path.replace("/",os.path.sep)

columns = ["total_cases","daily_cases","total_dead","daily_dead", "total_test","daily_test"]

dataset = pd.read_csv(path, index_col = 0)


array(['Scotland', 'Scotland', 'Scotland', ..., 'Wales', 'Wales', 'Wales'],
      dtype=object)

Grouping by data and indicator to sum the confirmed cases, deaths and tests among different regions

In [12]:
dataset = dataset[dataset.Country.values == "UK"]
dataset = dataset.groupby(["Date","Indicator"]).sum()

Creating a new dataframe with our specific format

In [14]:
uk_dataset = pd.DataFrame(index = dataset.index.levels[0] , columns = columns)
uk_dataset.reset_index()
uk_dataset = uk_dataset.fillna(0)

dataset

Value
Date       Indicator             
2020-01-25 ConfirmedCases       0
           Deaths               0
           Tests               31
2020-01-26 ConfirmedCases       0
           Deaths               0
...                           ...
2020-04-28 Deaths           25302
           Tests           599339
2020-04-29 ConfirmedCases  165221
           Deaths           26097
           Tests           632794

[288 rows x 1 columns]

Filling the new dataframe with the avaiable data

In [15]:
from datetime import datetime, timedelta
day = []
for days in dataset.index.levels[0]:
    try:
        #from 2nd row ---- to subtract a day from the date we had to convert it to a date and then back to a string
        yesterday = (datetime.strptime(days,"%Y-%m-%d")-timedelta(days=1)).strftime("%Y-%m-%d")
        uk_dataset.loc[days]["daily_cases"] = dataset.loc[days,"ConfirmedCases"]["Value"] - dataset.loc[yesterday,"ConfirmedCases"]["Value"]
        uk_dataset.loc[days]["daily_dead"] = dataset.loc[days,"Deaths"]["Value"] - dataset.loc[yesterday,"Deaths"]["Value"]
        uk_dataset.loc[days]["daily_test"] = dataset.loc[days,"Tests"]["Value"] - dataset.loc[yesterday,"Tests"]["Value"]
    except:
        #First row
        uk_dataset.loc[days]["daily_cases"] = dataset.loc[days,"ConfirmedCases"]["Value"]
        uk_dataset.loc[days]["daily_dead"] = dataset.loc[days,"Deaths"]["Value"]
        uk_dataset.loc[days]["daily_test"] = dataset.loc[days,"Tests"]["Value"]
    finally:
        #Single row
        uk_dataset.loc[days]["total_cases"] = dataset.loc[days,"ConfirmedCases"]["Value"]
        #Missing recoveres -> uk_dataset.loc[days]["actual_cases"] =
        uk_dataset.loc[days]["total_dead"] = dataset.loc[days,"Deaths"]["Value"]
        uk_dataset.loc[days]["total_test"] = dataset.loc[days,"Tests"]["Value"]



Converting dates from String to Datetime

In [16]:
import datetime as dt

index_list = uk_dataset.index.tolist()

for i in range(len(index_list)):
    index_list[i] = dt.datetime.strptime(index_list[i], '%Y-%m-%d').date()

uk_dataset.index = index_list
uk_dataset.index.name = 'date'

uk_dataset.tail()

,total_cases,daily_cases,total_dead,daily_dead,total_test,daily_test
date,,,,,,
2020-04-25,148377,4913,23635,843,517836,73614
2020-04-26,152840,4463,24055,420,543413,25577
2020-04-27,157149,4309,24393,338,569768,26355
2020-04-28,161145,3996,25302,909,599339,29571
2020-04-29,165221,4076,26097,795,632794,33455


In [17]:
#get path for os 
path = os.getcwd() + "/parsed_data/united_kingdom/"
path = path.replace("/",os.path.sep)
#save data as CSV 
uk_dataset.to_csv (path + "united_kingdom.csv", index = True, header=True)